In [59]:
import numpy as np
import theano
from theano import tensor as T
theano_rng = T.shared_randomstreams.RandomStreams(1234)
W_values = np.array([[1,1],[1,1]], dtype=theano.config.floatX)
bvis_values = np.array([1,1], dtype=theano.config.floatX)
bhid_values = np.array([1,1], dtype=theano.config.floatX)
W = theano.shared(W_values)
vbias = theano.shared(bvis_values)
hbias = theano.shared(bhid_values)

def propup(vis, v0_doc_len):
        pre_sigmoid_activation = T.dot(vis, W) + T.dot(hbias.reshape([1,hbias.shape[0]]).T,v0_doc_len).T        #---------------------------[edited]
        return [pre_sigmoid_activation, T.nnet.sigmoid(pre_sigmoid_activation)]

def sample_h_given_v(v0_sample, v0_doc_len):
    pre_sigmoid_h1, h1_mean = propup(v0_sample, v0_doc_len)
    h1_sample = theano_rng.binomial(size=h1_mean.shape,
                                         n=1, p=h1_mean,
                                         dtype=theano.config.floatX)
    return [pre_sigmoid_h1, h1_mean, h1_sample]

ipt = T.matrix()
ipt_rSum = ipt.sum(axis=1).reshape([1,ipt.shape[0]])

[out_1,out_2,out_3], updates =theano.scan( sample_h_given_v,
                                non_sequences =[ipt, ipt_rSum],
                                n_steps=1,
                                name="gibbs_hvh" )

hgv = theano.function( [ipt], outputs=[out_1,out_2,out_3])

In [60]:
b = np.array([[1,6,],[1,3],[5,1]], dtype = theano.config.floatX)

In [61]:
pre_sigmoid_ph, ph_mean, ph_sample = hgv(b)
print(pre_sigmoid_ph)
print(ph_mean)
print(ph_sample)

[[[ 14.  14.]
  [  8.   8.]
  [ 12.  12.]]]
[[[ 0.99999917  0.99999917]
  [ 0.99966466  0.99966466]
  [ 0.9999938   0.9999938 ]]]
[[[ 1.  1.]
  [ 1.  1.]
  [ 1.  1.]]]
